# Dislocation Quadrupoles
Dislocation Quadrupoles are structures designed with the goal of enabling the study of dislocation systems in periodic cells. In order for the system to be periodic, the cell must have a net Burgers vector of zero. To achieve this, we place two dislocations with Burgers vectors of equal magnitude and opposite sign (i.e. $+\textbf{b}$ and $-\textbf{b}$) in the structure.

The code is designed to allow the generation of these structures in as small a structure is required for a given separation of the dislocations along the glide direction.

To start, lets look at the $90^\circ$ partial dislocation in Diamond, using a dislocation cylinder approach documented in earlier docs and the same potential by [D. Holland and M. Marder](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.80.746):

In [1]:
from matscipy.dislocation import DiamondGlide90degreePartial, get_elastic_constants
from matscipy.calculators.manybody.explicit_forms.stillinger_weber import StillingerWeber,\
                                                                Holland_Marder_PRL_80_746_Si
from matscipy.calculators.manybody import Manybody
calc = Manybody(**StillingerWeber(Holland_Marder_PRL_80_746_Si))

alat, C11, C12, C44 = get_elastic_constants(calculator=calc, symbol="Si", verbose=False)


Si_disloc = DiamondGlide90degreePartial(alat, C11, C12, C44, symbol="Si")

bulk, disloc = Si_disloc.build_cylinder(radius=20)

Si_disloc.view_cyl(disloc, scale=0.3, add_bonds=True)


NGLWidget(layout=Layout(height='300px', width='100%'))

In [2]:
from matscipy.dislocation import CubicCrystalDislocationQuadrupole

quad = CubicCrystalDislocationQuadrupole(DiamondGlide90degreePartial, alat, C11, C12, C44, symbol="Si")

quad_bulk, quad_disloc = quad.build_quadrupole(glide_separation=4, self_consistent=False)

view = quad.view_quad(quad_disloc, scale=0.3, add_bonds=True)
view.control.zoom(0.2)
view

Periodic displacement iteration 0 -> max(|dr|) = 1.0393
Periodic displacement iteration 1 -> max(|dr|) = 0.1551
Periodic displacement iteration 2 -> max(|dr|) = 0.0062
Periodic displacement iteration 3 -> max(|dr|) = 0.0015
Periodic displacement iteration 4 -> max(|dr|) = 0.0006
Periodic displacements converged to r_tol=0.001 after 4 iterations


NGLWidget(layout=Layout(height='300px', width='100%'))

## Dislocation Glide in quadrupoles
As quadrupoles are extremely useful for modelling dislocations using plane-wave DFT, it can be convenient to be able to set up initial guesses for complex processes such as dislocation glide. In this scenario, we assume that the full infinite dislocation line glides in unison, ignoring the true "kink" process.

We can use the function `build_glide_quadrupoles` to construct a set of images for this system, which can optionally model the glide of either the "left" ($+\mathbf{b}$) or "right" ($-\mathbf{b}$) dislocation cores, or both at once.

In [3]:
num_images = 4

glide_quads = quad.build_glide_quadrupoles(nims=num_images, 
                                            glide_left=True, # Let the left dislocation glide
                                            glide_right=True, # Let the left dislocation glide
                                            self_consistent=False,
                                            glide_separation=4,
                                            verbose=False)

In [4]:
# image 0 will be identical to the above structure
quad.view_quad(glide_quads[1], scale=0.3, add_bonds=True)

NGLWidget(layout=Layout(height='300px', width='100%'))

In [5]:
quad.view_quad(glide_quads[2], scale=0.3, add_bonds=True)

NGLWidget(layout=Layout(height='300px', width='100%'))

In [6]:
quad.view_quad(glide_quads[3], scale=0.3, add_bonds=True)

NGLWidget(layout=Layout(height='300px', width='100%'))

## Quadrupole Kink
Dislocation kink is commonly a more energetically accessible way for a dislocation to move in the glide direction. Rather than the full dislocation line gliding as one, the kink mechanism involves a small section of the dislocation line nucleating out by a glide vector, forming a pair of kinks. This pair of kinks can then diffuse along the direction of the dislocation line, advancing the line locally by a single glide vector.

In the quadrupole cells, we model an infinite chain of single kinks, where the dislocation line advances by one glide direction every periodic repetition of the structure in the Z direction. This is distinct from the kink-pair mechanism, which would need two kinks (which migrate the dislocation line in opposite directions), however the single kink structures can be useful in making kink behaviour accessible to the small scales required by expensive methods such as DFT.

To actually construct kink structures from a quadrupole object, we use the `build_kink_quadrupole()` method of `CubicCrystalDislocationQuadrupole`. The main two important parameters to define are `invert_direction`, which changes the directionality of the kinks (some systems have distinct "left" kinks and "right" kinks), and `z_reps`, which both changes the size of the final structure, as well as the initial guess for the length of the kink.

:::{note}
Because the kink advances the system by a single glide vector, and cells are typically multiple glide vectors long, the structure will not initially be periodic. To solve this, some atomic layers are removed (as controlled by `layer_tol`). In multispecies systems, this may disrupt the atomic composition of the end structure.
:::

In [8]:
kink_quad = quad.build_kink_quadrupole(z_reps=5, 
                                       self_consistent=False,
                                       glide_separation=4,
                                       verbose=False, layer_tol=1e-2)

quad.view_quad(kink_quad, scale=0.6, add_bonds=False)

1
432


NGLWidget(layout=Layout(height='300px', width='100%'))